# Triple A - Group Project
## Predicting Taxi Demand in spatial and time resolution

In this notebook we will focus on the SVM modelling based on the different temporal and geographical resolutions. We start off doing final adjustments to prepare data for prediction and then start with a linear SVM predicting on hourly inputs on Census Tract level.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
census_hourly = pd.read_csv('data/census_hourly.csv')
census_hourly.head()

,Trip Hour,Tract,trip_count,app_temp,clouds,dewpt,precip,pres,rh,snow,...,wind_dir,wind_gust_spd,wind_spd,nighttime,poi_count,is_holiday,isWeekend,Trip Start Day,Trip Start Hour,Trip Start Month
0,2024-01-01 00:00:00,17031842400,0.000000,-5.7,87.0,-1.5,0.0,997.0,85.0,0.0,...,320.0,12.4,8.8,1.0,0,1,0,1,0,1
1,2024-01-01 00:00:00,17031840300,0.000000,-5.7,87.0,-1.5,0.0,997.0,85.0,0.0,...,320.0,12.4,8.8,1.0,3,1,0,1,0,1
2,2024-01-01 00:00:00,17031841100,0.000000,-5.7,87.0,-1.5,0.0,997.0,85.0,0.0,...,320.0,12.4,8.8,1.0,57,1,0,1,0,1
3,2024-01-01 00:00:00,17031841200,0.090909,-5.7,87.0,-1.5,0.0,997.0,85.0,0.0,...,320.0,12.4,8.8,1.0,3,1,0,1,0,1
4,2024-01-01 00:00:00,17031839000,4.800000,-5.7,87.0,-1.5,0.0,997.0,85.0,0.0,...,320.0,12.4,8.8,1.0,17,1,0,1,0,1


To end up with interpretable time data for the algorithm, we need to adjust it in a cyclic way using sin and cos transformations. This way the algorithm will be able to compute sensible distances between eg. hours 23 and 1 vs hour 1 and 3.

In [3]:
# Hour of the day
census_hourly['Trip Start Hour'] = pd.to_datetime(census_hourly['Trip Hour']).dt.hour

census_hourly["Trip Start Hour Sin"] = np.sin(2 * np.pi * census_hourly["Trip Start Hour"] / 24)
census_hourly["Trip Start Hour Cos"] = np.cos(2 * np.pi * census_hourly["Trip Start Hour"] / 24)


# Day of the Week
census_hourly['Trip Start Day'] = pd.to_datetime(census_hourly['Trip Hour']).dt.weekday

census_hourly["Trip Start Day Sin"] = np.sin(2 * np.pi * census_hourly["Trip Start Day"] / 7)
census_hourly["Trip Start Day Cos"] = np.cos(2 * np.pi * census_hourly["Trip Start Day"] / 7)


# Month of the Year
census_hourly['Trip Start Month'] = pd.to_datetime(census_hourly['Trip Hour']).dt.month

census_hourly["Trip Start Month Sin"] = np.sin(2 * np.pi * census_hourly["Trip Start Month"] / 12)
census_hourly["Trip Start Month Cos"] = np.cos(2 * np.pi * census_hourly["Trip Start Month"] / 12)


census_hourly.head(5)

,Trip Hour,Tract,trip_count,app_temp,clouds,dewpt,precip,pres,rh,snow,...,isWeekend,Trip Start Day,Trip Start Hour,Trip Start Month,Trip Start Hour Sin,Trip Start Hour Cos,Trip Start Day Sin,Trip Start Day Cos,Trip Start Month Sin,Trip Start Month Cos
0,2024-01-01 00:00:00,17031842400,0.000000,-5.7,87.0,-1.5,0.0,997.0,85.0,0.0,...,0,0,0,1,0.0,1.0,0.0,1.0,0.5,0.866025
1,2024-01-01 00:00:00,17031840300,0.000000,-5.7,87.0,-1.5,0.0,997.0,85.0,0.0,...,0,0,0,1,0.0,1.0,0.0,1.0,0.5,0.866025
2,2024-01-01 00:00:00,17031841100,0.000000,-5.7,87.0,-1.5,0.0,997.0,85.0,0.0,...,0,0,0,1,0.0,1.0,0.0,1.0,0.5,0.866025
3,2024-01-01 00:00:00,17031841200,0.090909,-5.7,87.0,-1.5,0.0,997.0,85.0,0.0,...,0,0,0,1,0.0,1.0,0.0,1.0,0.5,0.866025
4,2024-01-01 00:00:00,17031839000,4.800000,-5.7,87.0,-1.5,0.0,997.0,85.0,0.0,...,0,0,0,1,0.0,1.0,0.0,1.0,0.5,0.866025


In [4]:
# everything in float or int type, only need to drop Trip Hour column 
census_hourly = census_hourly.drop(columns =['Trip Hour'])
census_hourly = census_hourly.astype(float)
#census_hourly.dtypes

Tract                     int64
trip_count              float64
app_temp                float64
clouds                  float64
dewpt                   float64
precip                  float64
pres                    float64
rh                      float64
snow                    float64
temp                    float64
uv                      float64
vis                     float64
wind_dir                float64
wind_gust_spd           float64
wind_spd                float64
nighttime               float64
poi_count                 int64
is_holiday                int64
isWeekend                 int64
Trip Start Day            int32
Trip Start Hour           int32
Trip Start Month          int32
Trip Start Hour Sin     float64
Trip Start Hour Cos     float64
Trip Start Day Sin      float64
Trip Start Day Cos      float64
Trip Start Month Sin    float64
Trip Start Month Cos    float64
dtype: object

In [5]:
census_hourly.isna().sum()

Tract                   0
trip_count              0
app_temp                0
clouds                  0
dewpt                   0
precip                  0
pres                    0
rh                      0
snow                    0
temp                    0
uv                      0
vis                     0
wind_dir                0
wind_gust_spd           0
wind_spd                0
nighttime               0
poi_count               0
is_holiday              0
isWeekend               0
Trip Start Day          0
Trip Start Hour         0
Trip Start Month        0
Trip Start Hour Sin     0
Trip Start Hour Cos     0
Trip Start Day Sin      0
Trip Start Day Cos      0
Trip Start Month Sin    0
Trip Start Month Cos    0
dtype: int64

In [15]:
X = census_hourly.drop(columns=["trip_count"])
y = census_hourly["trip_count"]

numeric_feats = X.select_dtypes(include=["float64", "int32", "float64"]).columns.drop("Tract", errors="ignore")
categorical_feats = ["Tract"]

numeric_transformer = Pipeline([
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_feats),
    ("cat", categorical_transformer, categorical_feats)
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Due to resource restrictions we don't use true kernel svm but use linear svr and kernel approximation since those scale linearly in n.

In [9]:
from sklearn.svm import LinearSVR

pipeline_lin_svr = Pipeline([
    ("preproc", preprocessor),
    ("svr", LinearSVR(
        loss="squared_epsilon_insensitive",          # classic SVR loss squared for dual = False
        dual=False,                          # one formulation that often converges faster
        max_iter=100_000,                     # you may need to bump this up
        tol=1e-3,                            # convergence tolerance
        random_state=42
    ))
])

pipeline_lin_svr.fit(X_train, y_train)
y_pred_lin = pipeline_lin_svr.predict(X_test)

# metrics
mse_lin = mean_squared_error(y_test, y_pred_lin)
mae_lin = mean_absolute_error(y_test, y_pred_lin)
r2_lin  = r2_score(y_test, y_pred_lin)

print(f"Linear SVR →  MSE: {mse_lin:.2f}, MAE: {mae_lin:.2f},  R²: {r2_lin:.3f}")

Linear SVR →  MSE: 16.51, MAE: 0.83,  R²: 0.531


In [9]:
#pipeline_lin = Pipeline([
#    ("preproc", preprocessor),
#    ("svr", SVR(kernel="linear"))
#])
#
#pipeline_lin.fit(X_train, y_train)
#y_pred_lin = pipeline_lin.predict(X_test)
#
## metrics
#mse_lin = mean_squared_error(y_test, y_pred_lin)
#mae_lin = mean_absolute_error(y_test, y_pred_lin)
#r2_lin  = r2_score(y_test, y_pred_lin)
#
#print(f"Linear SVR →  MSE: {mse_lin:.2f}, MAE: {mae_lin:.2f},  R²: {r2_lin:.3f}")


In [16]:
from sklearn.kernel_approximation import RBFSampler

pipeline_rbf_approx = Pipeline([
    ("preproc", preprocessor),
    ("rbf_features", RBFSampler(
        gamma=0.1,                             # RBF bandwidth
        n_components=500,                      # number of random features; tune this
        random_state=42
    )),
    ("svr", LinearSVR(
        loss="squared_epsilon_insensitive",
        dual=False,
        max_iter=50_000,
        tol=1e-3,
        random_state=42
    ))
])

pipeline_rbf_approx.fit(X_train, y_train)
y_pred_rbf = pipeline_rbf_approx.predict(X_test)

# metrics
mse_rbf = mean_squared_error(y_test, y_pred_rbf)
mae_rbf = mean_absolute_error(y_test, y_pred_rbf)
r2_rbf  = r2_score(y_test, y_pred_rbf)

print(f"Linear SVR →  MSE: {mse_rbf:.2f}, MAE: {mae_rbf:.2f},  R²: {r2_rbf:.3f}")

MemoryError: Unable to allocate 26.1 GiB for an array with shape (7013556, 500) and data type float64

In [ ]:
## RBF
#pipeline_rbf = Pipeline([
#    ("preproc", preprocessor),
#    ("svr", SVR(kernel="rbf"))
#])
#pipeline_rbf.fit(X_train, y_train)
#y_pred_rbf = pipeline_rbf.predict(X_test)
#
## Poly
#pipeline_poly = Pipeline([
#    ("preproc", preprocessor),
#    ("svr", SVR(kernel="poly", degree=3))  # degree=3 default
#])
#pipeline_poly.fit(X_train, y_train)
#y_pred_poly = pipeline_poly.predict(X_test)
#
## Evaluate both
#for name, y_pred in [("RBF", y_pred_rbf), ("Poly", y_pred_poly)]:
#    mse = mean_squared_error(y_test, y_pred)
#    mae = mean_absolute_error(y_test, y_pred)
#    r2  = r2_score(y_test, y_pred)
#    print(f"{name} SVR →  MSE: {mse:.2f}, MAE: {mae:.2f}, R²: {r2:.3f}")
#

In [ ]:
## parameter grid
#param_grid = {
#    "svr__kernel": ["linear", "rbf", "poly"],
#    "svr__C":      [0.1, 1, 10],
#    "svr__gamma":  ["scale", "auto"],
#    "svr__degree": [2, 3, 4]   # only used when kernel="poly"
#}
#
#grid = GridSearchCV(
#    Pipeline([("preproc", preprocessor), ("svr", SVR())]),
#    param_grid,
#    cv=5,
#    scoring="neg_mean_squared_error",
#    verbose=2,
#    n_jobs=-1
#)
#
#grid.fit(X_train, y_train)
#
#print("Best params:", grid.best_params_)
#print("Best CV MSE:", -grid.best_score_)
#

In [ ]:
#best_model = grid.best_estimator_
#y_pred_best = best_model.predict(X_test)
#
#mse_best = mean_squared_error(y_test, y_pred_best)
#mae_best = mean_absolute_error(y_test, y_pred_best)
#r2_best  = r2_score(y_test, y_pred_best)
#
#print(f"Tuned SVR →  MSE: {mse_best:.2f}, MAE: {mae_best:.2f}, R²: {r2_best:.3f}")
